In [4]:
import cv2
from os.path import dirname, join
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream as vs
import numpy as np
import imutils
import time
import cv2
import os

protoPath = join('C:/Users/PROMIT/Desktop/contents/Face-Mask-Detection/face_detector/', 'deploy.prototxt')
modelPath = join('C:/Users/PROMIT/Desktop/contents/DFT/AI_ML/', "res10_300x300_ssd_iter_140000.caffemodel")
#Loading face detection classifier
face_detection_classifier=cv2.dnn.readNetFromCaffe(protoPath,modelPath)

In [2]:
from tensorflow.keras.models import load_model
mask_detector=load_model(r'C:\Users\PROMIT\Desktop\Jupyter_notebook_files\Mask_detector\MY_MASK_DETECTOR')

In [5]:
#function to detect faces and predict 
def face_det_mask_pred(frame,face_detection_classifier,mask_detector):
    (img_height,img_width)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(224,224),(104.0,177.0,123.0))
    face_detection_classifier.setInput(blob)
    face_detections=face_detection_classifier.forward()
    
    faces=[]
    locs=[]
    preds=[]
    num_faces=face_detections.shape[2]
    for i in range(num_faces):
        confidence=face_detections[0,0,i,2]
        if confidence>0.5:
            box=face_detections[0,0,i,3:7]*np.array([img_width,img_height,img_width,img_height])
            (startX,startY,endX,endY)=box.astype("int")
            face=frame[startY:endY,startX:endX]
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            faces.append(face)
            locs.append((startX,startY,endX,endY))
    if len(faces)>0:
        faces=np.array(faces,dtype="float32")
        preds=mask_detector.predict(faces)
        
      
    return (locs,preds)

In [7]:
cap=vs(src=0).start()
while True:
  
    #Capture frame by frame
    image=cap.read()
    img_height,img_width=image.shape[:2]
    (locs,preds)=face_det_mask_pred(image,face_detection_classifier,mask_detector)
    for (box,pred) in zip(locs,preds):
        (startX,startY,endX,endY)=box
        (mask,withoutMask)=pred 
        if mask>withoutMask:
            label="Mask"
            temp_color=(51,255,102)
        else:
            label="Without Mask"
            temp_color=(255,255,0)
            
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,1,temp_color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),(255,0,0),2)
    cv2.namedWindow("output",cv2.WINDOW_NORMAL)
    cv2.imshow("output",image)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

         
cv2.destroyAllWindows()

